# Collaborative Filtering - memory based

user-item matrix를 이용해 플레이리스트에 들어있는 노래를 보고 유사한 플레이리스트를 추천해줄 것이다.


In [1]:
import json
import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.neighbors import NearestNeighbors

## 데이터 기본 전처리

#### 플레이리스트
* 최대 인덱스 : 153428
* 인덱스 갯수 : 115071

#### 노래
* 최대 인덱스 : 707988 
* 인덱스 갯수 : 615142

이렇게 인덱스 갯수와 최대 인덱스가 다르다.

이런 데이터를 그냥 넣고 만들면 user-item matrix로 변환되면서 중간에 비어 있던 인덱스가 붙어서 매트릭스가 만들어지기에<br/>
입력값을 만들 때 item id의 max만큼의 배열을 잡으면 크기가 안맞게 된다.


예를들면<br/>
[1,5,12,64] 가 있다면  64크기의 배열이 아닌 4크기의 배열로 잡아야한다.<br/>
또한 1 -> 0, 5 -> 1, 12 -> 2, 64 -> 3로 인덱스에 변화가 생긴다. 이를 맞춰주기 위한 데이터 작업을 선행해야 한다.


위와 같은 문제가 song에서도나고, plylst에서도 난다.<br/>
song과 plylst의 max와 nunique가 다르기때문에 발생하는 문제

이를 해결하기 위해 user-item matrix의 크기를 [플레이리스트 최대 인덱스, 노래 최대 인덱스]로 해줘야 한다.



In [2]:
train = pd.read_json("./data/train.json", typ='frame')

In [3]:
plylst_song_map = train[['id','songs']]

plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)
# plylst_song_map['include'] = True

plylst_song_map = plylst_song_map.rename(columns = {'id' : 'plylst_id' , 'songs' : 'song_id'})
plylst_song_map_withnan = plylst_song_map.rename(columns = {'id' : 'plylst_id' , 'songs' : 'song_id'})

# del plylst_song_map_unnest

In [4]:
plylst_song_map_withnan['plylst_id'] = plylst_song_map_withnan['plylst_id'].astype(int)

all_plylst = pd.DataFrame(list(range(0, plylst_song_map_withnan['plylst_id'].max() + 1)), columns=['plylst_id'])

In [5]:
present_plylst_song = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map_withnan.columns)

In [6]:
all_plylst = pd.merge(all_plylst, present_plylst_song, on ='plylst_id', how='left')

In [7]:
print(len(all_plylst) - len(present_plylst_song))
print(all_plylst['plylst_id'].min(), all_plylst['plylst_id'].max())
print(plylst_song_map_withnan['plylst_id'].max() - plylst_song_map_withnan['plylst_id'].nunique())
print(plylst_song_map_withnan['plylst_id'].min(), plylst_song_map_withnan['plylst_id'].max())

38358
0 153428
38357
1 153428


In [8]:
all_plylst[all_plylst['song_id'].isnull()].head()

,plylst_id,song_id
0,0,NaN
22,3,NaN
290,10,NaN
291,11,NaN
475,17,NaN


In [9]:
plylst_song_map_withnan['song_id'] = plylst_song_map_withnan['song_id'].astype(int)

all_song = pd.DataFrame(list(range(0, plylst_song_map_withnan['song_id'].max() + 1)), columns=['song_id'])


In [10]:
all_song = pd.merge(all_song, present_plylst_song, on ='song_id', how='left')

In [11]:
print(len(all_song) - len(present_plylst_song))
print(all_song['song_id'].min(), all_song['song_id'].max(), all_song['song_id'].nunique())

print(plylst_song_map_withnan['song_id'].max() - plylst_song_map_withnan['song_id'].nunique())
print(plylst_song_map_withnan['song_id'].min(), plylst_song_map_withnan['song_id'].max(), plylst_song_map_withnan['song_id'].nunique())

92847
0 707988 707989
92846
0 707988 615142


In [12]:
len(plylst_song_map)

5285871

In [13]:
plylst_song_map_withnan = pd.concat([all_plylst,all_song]).drop_duplicates(subset = ['plylst_id','song_id'])


In [14]:
plylst_song_map_withnan['include'] = True

plylst_song_map_withnan['include'] = ~plylst_song_map_withnan.isnull().any(axis=1)   # Nan이 있으면 False
plylst_song_map_withnan['song_id'] = plylst_song_map_withnan['song_id'].fillna(1)
plylst_song_map_withnan['plylst_id'] = plylst_song_map_withnan['plylst_id'].fillna(1)

plylst_song_map_withnan

,plylst_id,song_id,include
0,0.0,1.0,False
1,1.0,47805.0,True
2,1.0,308020.0,True
3,1.0,662131.0,True
4,1.0,418970.0,True
...,...,...,...
5378607,1.0,707962.0,False
5378649,1.0,707968.0,False
5378677,1.0,707971.0,False
5378679,1.0,707973.0,False


In [15]:
plylst_song_map['plylst_id'] = plylst_song_map['plylst_id'].astype(int)
plylst_song_map['song_id'] = plylst_song_map['song_id'].astype(int)

print(plylst_song_map['plylst_id'].max(), plylst_song_map['plylst_id'].nunique())
print(plylst_song_map['song_id'].max(), plylst_song_map['song_id'].nunique())

153428 115071
707988 615142


# to User-Item matrix (sparse)

아이템 기반 추천이 더 성능이 좋다고 한ek


item-user matrix가 (153428, 707989)가 된 이유는 원래 데이터에 song_id는 0부터 시작했지만 plylst_id는 1부터 시작했따.<br/>
그래서 0이 추가되어 max값 보다 하나가 더 늘어남

In [16]:
from scipy import sparse

def create_matrix(data, user_col, item_col, rating_col):

    rows = data[user_col].astype('category').cat.codes
    cols = data[item_col].astype('category').cat.codes
    rating = data[rating_col]
    ratings = sparse.csr_matrix((rating, (rows, cols)))
#     ratings.eliminate_zeros()
    return ratings, data

sparse_rating, data = create_matrix(plylst_song_map_withnan, 'plylst_id', 'song_id', "include")
# movieId, userId순으로 하면 열에 movie, 행에 사람들의 평가가 들어간 item-user 매트릭스

# dense_rating = sparse_rating.toarray()
# dense_rating = dense_rating.astype(np.uint8) # bool --> uint8
# print(dense_rating.shape)

print("(아이템 수, 유저 수) :",sparse_rating.shape)

# 저장하기 save_sparse_csr('/users/nickbecker/Python_Projects/lastfm_sparse_artist_matrix_binary.npz', wide_artist_data_zero_one_sparse)

(아이템 수, 유저 수) : (153429, 707989)


In [17]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm ='brute')
model_knn.fit(sparse_rating)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
song_size = sparse_rating.shape[1]
history = np.zeros((1,song_size))

In [19]:
print("all data :",sparse_rating.shape)
print("input data :",history.shape)

all data : (153429, 707989)
input data : (1, 707989)


In [20]:
plylstTosong = train[['id','songs']]

In [21]:
query_idx = np.random.choice(sparse_rating.shape[0]) # 빈 리스트가 올 수도 있으니 바꿔줘야해

history = np.zeros((1,song_size))
# print(query_idx)
# print(len(plylst_song_map.loc[plylst_song_map['plylst_id'] == query_idx]))
# print(plylstTosong.loc[plylstTosong['id'] == query_idx]['songs'])
# print(type(plylstTosong.loc[plylstTosong['id'] == query_idx]['songs']))

input_songs = plylstTosong.loc[plylstTosong['id'] == query_idx]['songs'].tolist()

if input_songs : 
    history[0][input_songs] = 1

    print(sum(history[0]))
    distances, indices = model_knn.kneighbors(history, n_neighbors = 6)

    for i in range(0, len(distances.flatten())) :
        if i == 0 :
            print("Recommendations for movieId:{0}".format(query_idx))
        else :
            print("{0} : {1}, with distance of {2}".format(i, indices.flatten()[i], distances.flatten()[i]))
        

37.0
Recommendations for movieId:41663
1 : 147168, with distance of 0.6203368016990004
2 : 33070, with distance of 0.629821977137059
3 : 107867, with distance of 0.7152526012742503
4 : 95806, with distance of 0.75145209359952
5 : 150828, with distance of 0.7794356133718576


/home/junho/anaconda3/envs/junho/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':


-----------------------------------------------------

In [ ]:
plylst_song_map['song_id'] = plylst_song_map['song_id'].astype(int)

print(plylst_song_map['song_id'].nunique(), plylst_song_map['song_id'].max())

In [ ]:
plylst_song_map['song_id'] = plylst_song_map['song_id'].astype(str)

plylst_song_map_ex = plylst_song_map.groupby(['plylst_id'])['song_id'].apply(','.join).reset_index()

plylst_song_map_ex['song_id'] = plylst_song_map_ex['song_id'].apply(lambda x : eval("["+x+"]"))

== train

In [ ]:
unique_song_id = list(sorted(map(int,plylst_song_map['song_id'].unique())))
idx_list = list(range(len(unique_song_id)))

IdxToRealidx = dict(zip(unique_song_id,idx_list))

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm ='brute')
model_knn.fit(sparse_rating)

In [ ]:
song_size = plylst_song_map['song_id'].nunique()
history = np.zeros((1,song_size))

In [ ]:
song_meta = pd.read_json('./data/song_meta.json', typ='frame')
song_id_name = song_meta[['id','song_name']]
SongidToName = dict(zip(song_meta['id'],song_meta['song_name']))

In [ ]:
# make input data
for query_idx in range(sparse_rating.shape[0]) :
#     query_idx = np.random.choice(sparse_rating.shape[0])

    history = np.zeros((1,song_size))
    print(query_idx)
    for i in plylst_song_map_ex.loc[plylst_song_map_ex['plylst_id'] == query_idx]['song_id'] :

        history[0][IdxToRealidx[i]] = 1

    # inference     
    distances, indices = model_knn.kneighbors(history, n_neighbors = 6)

#     for i in range(0, len(distances.flatten())) :
#         if i == 0 :
#             print("Recommendations for movieId:{0}".format(SongidToName[query_idx]))
#         else :
#             print("{0} : {1}, with distance of {2}".format(i, SongidToName[indices.flatten()[i]], distances.flatten()[i]))
        